# Factor Component Analysis

Haytham Mohamed - INFS890 - Spring 2020

Features reduction using FCA for order data. Using factor_analyzer.factor_analyzer module

In [326]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn import preprocessing
from datetime import datetime
from scipy import stats
from sklearn.decomposition import PCA

sns.set()
sns.set(color_codes=True)
#sns.set_color_codes()

pd.options.display.max_rows = 15
pd.options.display.float_format = '{:,.3f}'.format

In [327]:
# 1- read processed file

home_dir = '/Users/hmohamed/github/data-research-spring2020/sock-shop'

file_dir = home_dir + '/processed-data/'
data_file = 'order_flow_standardized_data.csv'

save=True


In [328]:
def read_df(file_dir, data_file):
    df = pd.read_csv(file_dir + data_file)
    return to_time_series(df)

def to_time_series(df, index_col_name='date'):
    df[index_col_name] = pd.to_datetime(df[index_col_name])
    df.set_index(index_col_name, inplace=True)
    df.sort_index(inplace=True)
    return df

def merge(df, series):
    return pd.merge_asof(df, series, left_index=True, right_index=True, tolerance=pd.Timedelta('1 second')).bfill()    

In [329]:
normalized_data = read_df(file_dir, data_file)
normalized_data.head(5) 

,front-end_cpu_use,orders_cpu_use,user_cpu_use,user-db_cpu_use,shipping_cpu_use,payment_cpu_use,carts_cpu_use,carts-db_cpu_use,front-end_pods,orders_pods,orders-db_pods,user_pods,user-db_pods,shipping_pods,payment_pods,carts_pods,carts-db_pods,nodes_cpu_use,orders_req,orders_ltcy
date,,,,,,,,,,,,,,,,,,,,
2020-03-26 19:46:22,-1.292,-1.529,-1.401,-1.360,-0.956,-1.791,-0.869,-0.883,-1.602,-1.613,0.000,-1.613,0.000,-1.613,-1.613,-1.613,0.000,-0.770,-1.163,0.240
2020-03-26 19:46:37,-1.243,-1.453,-1.327,-1.236,-0.956,-1.791,-0.608,-0.828,-1.602,-1.613,0.000,-1.613,0.000,-1.613,-1.613,-1.613,0.000,-1.063,-1.163,0.240
2020-03-26 19:46:52,-1.227,-1.378,-1.253,-1.236,-0.956,-1.791,-0.156,-0.818,-1.602,-1.613,0.000,-1.613,0.000,-1.613,-1.613,-1.613,0.000,-1.322,-1.163,0.240
2020-03-26 19:47:07,-1.227,-1.378,-1.327,-1.236,-0.956,-1.791,-0.521,-0.833,-1.602,-1.613,0.000,-1.613,0.000,-1.613,-1.613,-1.613,0.000,-1.365,-1.168,0.240
2020-03-26 19:47:22,-1.227,-1.453,-1.327,-1.236,-0.956,-1.791,-0.503,-0.848,-1.602,-1.613,0.000,-1.613,0.000,-1.613,-1.613,-1.613,0.000,-1.356,-1.113,0.240


In [330]:
normalized_data.shape

(6756, 20)

In [331]:
targets = normalized_data['orders_ltcy']
standardized_inputs = normalized_data.drop(['orders_ltcy'], axis=1)
cols = standardized_inputs.columns.values

standardized_inputs.shape


(6756, 19)

# Multicollinearity

Use Variance Inflation Factor (VIF) from the statmodels. VIF measures how big is the square root of the standard error is compared to the case there is no multicollinearity between the variables. 
Conventionally:

$VIF = 1$ means no multicollinearity

$1< VIF < 5$ perfectly okay

$10 < VIF$ unacceptable range (some times < 6 or 8)

In [332]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

standardized_inputs = pd.DataFrame(standardized_inputs, columns=cols)
vif_inputs = standardized_inputs

vif_inputs = vif_inputs.drop('user_cpu_use', 1)
vif_inputs = vif_inputs.drop('front-end_cpu_use', 1)
vif_inputs = vif_inputs.drop('user-db_cpu_use', 1)


vif = pd.DataFrame()
vif['features'] = [col for col in vif_inputs.columns.values]
vif['VIF'] = [variance_inflation_factor(vif_inputs.values,i) for i in range(vif_inputs.shape[1])]
vif

/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


,features,VIF
0,orders_cpu_use,5.180
1,shipping_cpu_use,1.325
2,payment_cpu_use,6.027
3,carts_cpu_use,3.852
4,carts-db_cpu_use,5.069
...,...,...
11,payment_pods,inf
12,carts_pods,inf
13,carts-db_pods,nan
14,nodes_cpu_use,8.582


In [333]:
standardized_inputs = standardized_inputs.drop('user_cpu_use', 1)
standardized_inputs = standardized_inputs.drop('front-end_cpu_use', 1)
standardized_inputs = standardized_inputs.drop('user-db_cpu_use', 1)

vif values are OK. Consider using PCA or apply feature selection method to reduce dimentionality when using a ML model.

# Correlation Matrix

In [334]:
corr = standardized_inputs.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(3)

/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/matplotlib/colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/pandas/io/formats/style.py:1089: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/pandas/io/formats/style.py:1090: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


,orders_cpu_use,shipping_cpu_use,payment_cpu_use,carts_cpu_use,carts-db_cpu_use,front-end_pods,orders_pods,orders-db_pods,user_pods,user-db_pods,shipping_pods,payment_pods,carts_pods,carts-db_pods,nodes_cpu_use,orders_req
orders_cpu_use,1.000,0.181,0.012,0.679,0.422,-0.184,-0.187,nan,-0.187,nan,-0.187,-0.187,-0.187,nan,0.783,0.819
shipping_cpu_use,0.181,1.000,0.225,-0.028,0.181,0.177,0.177,nan,0.177,nan,0.177,0.177,0.177,nan,0.087,0.006
payment_cpu_use,0.012,0.225,1.000,0.028,0.236,0.832,0.833,nan,0.833,nan,0.833,0.833,0.833,nan,0.180,0.098
carts_cpu_use,0.679,-0.028,0.028,1.000,0.047,-0.008,-0.008,nan,-0.008,nan,-0.008,-0.008,-0.008,nan,0.570,0.665
carts-db_cpu_use,0.422,0.181,0.236,0.047,1.000,-0.124,-0.127,nan,-0.127,nan,-0.127,-0.127,-0.127,nan,0.735,0.553
front-end_pods,-0.184,0.177,0.832,-0.008,-0.124,1.000,0.999,nan,0.999,nan,0.999,0.999,0.999,nan,-0.160,-0.121
orders_pods,-0.187,0.177,0.833,-0.008,-0.127,0.999,1.000,nan,1.000,nan,1.000,1.000,1.000,nan,-0.162,-0.125
orders-db_pods,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
user_pods,-0.187,0.177,0.833,-0.008,-0.127,0.999,1.000,nan,1.000,nan,1.000,1.000,1.000,nan,-0.162,-0.125
user-db_pods,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [335]:
def plot_corr(df,size=10):
    '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''

    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns);
    plt.yticks(range(len(corr.columns)), corr.columns);
    
#plot_corr(standardized_inputs,20)    

The function most_highly_correlated() will print out the linear correlation coefficients for each pair of variables in your data set, in order of the correlation coefficient. This lets you see very easily which pair of variables are most highly correlated.

In [336]:
def most_highly_correlated(mydataframe, numtoreport):
    # find the correlations
    cormatrix = mydataframe.corr()
    # set the correlations on the diagonal or lower triangle to zero,
    # so they will not be reported as the highest ones:
    cormatrix *= np.tri(*cormatrix.values.shape, k=-1).T
    # find the top n correlations
    cormatrix = cormatrix.stack()
    cormatrix = cormatrix.reindex(cormatrix.abs().sort_values(ascending=False).index).reset_index()
    # assign human-friendly names
    cormatrix.columns = ["FirstVariable", "SecondVariable", "Correlation"]
    return cormatrix.head(numtoreport)

mcdf = most_highly_correlated(standardized_inputs, 30)
mcdf = mcdf[mcdf.Correlation > 0.5]

mcdf

,FirstVariable,SecondVariable,Correlation
0,payment_pods,carts_pods,1.000
1,orders_pods,shipping_pods,1.000
2,orders_pods,carts_pods,1.000
3,orders_pods,user_pods,1.000
4,user_pods,shipping_pods,1.000
...,...,...,...
24,carts-db_cpu_use,nodes_cpu_use,0.735
25,orders_cpu_use,carts_cpu_use,0.679
26,carts_cpu_use,orders_req,0.665
27,carts_cpu_use,nodes_cpu_use,0.570


# Component Factor Analysis

### Adequacy Test

Check adquacy of the correlation matrix for factor analysis. Checking both Bartlett's test if the correlation matrix is collectively significant to include factors, and checking the Measure of Adquacy Samplin (MSA), a.k.a Kaiser-Meyer-Olkin (KMO) to check the pattern

In [337]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

# Bartlett ’s test
chi_square_value,p_value=calculate_bartlett_sphericity(standardized_inputs)

chi_square_value, p_value

/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/numpy/linalg/linalg.py:2116: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


(nan, nan)

In this Bartlett’s test, the p-value is 0. The test was statistically significant, indicating that the observed correlation matrix is not an identity matrix.

In [338]:
from factor_analyzer.factor_analyzer import calculate_kmo

kmo_all,kmo_model=calculate_kmo(standardized_inputs)

kmo_model

/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/factor_analyzer/utils.py:248: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn('The inverse of the variance-covariance matrix '
/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/factor_analyzer/utils.py:205: RuntimeWarning: divide by zero encountered in true_divide
  Is = np.sqrt(1 / np.diag(m))
/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/factor_analyzer/utils.py:206: RuntimeWarning: invalid value encountered in multiply
  retval = Is * m * np.repeat(Is, numrows).reshape(numrows, numrows)


nan

The overall KMO for our data is 0.785, which is excellent ( > 0.5). This value indicates that you can proceed with your planned factor analysis.

In [339]:
kmo_all

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan])

Features KMO values are significant ( > 0.5). It would be ok to consider them all to analyze the factors.

### Number of Factors

For choosing the number of factors, you can use the Kaiser criterion and scree plot. Both are based on eigenvalues.

In [340]:
# Create factor analysis object and perform factor analysis
from factor_analyzer.factor_analyzer import FactorAnalyzer

fa = FactorAnalyzer(n_factors=25, rotation=None)
fa.fit(standardized_inputs)
# Check Eigenvalues
ev, v = fa.get_eigenvalues()
ev

/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/factor_analyzer/utils.py:91: RuntimeWarning: invalid value encountered in true_divide
  x = (x - x.mean(0)) / x.std(0)


ValueError: array must not contain infs or NaNs

Here, you can see only for 2 factors eigenvalues are greater than one. It means we need to choose only 2 factors (or unobserved variables). Also we can draw a scree plot to check how many components to consider. Number of factors will be for the one spotted with Eigenvalue >= 1

In [ ]:
# Create scree plot using matplotlib
plt.scatter(range(1,standardized_inputs.shape[1]+1),ev)
plt.plot(range(1,standardized_inputs.shape[1]+1),ev)
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.grid()
plt.show()

### Component Factor Loadings

Choosing 2 factors however segregate the features into three interpreted (logical) factors. Looking the loading below we see the features loaded as follows:

Factor 1:  svc_cpu_use, svc_cpu_thr, svc_net_use, svc_disk_use, system_net_use, svc_req_size, svc_resp_size, svc_req_rate ==> a factor about svc_usage_tolerance

Factor 2: system_cpu_use and system_cpu_sat, svc_pods ==> a factor about system_workload_tolerance

The new components are just the two main dimensions of variation


In [ ]:
number_of_factors = 4
fa = FactorAnalyzer(n_factors=number_of_factors, rotation="promax")  # promax rotation eliminate cross loadings
fa.fit(standardized_inputs)

factors_heading = []
for i in range(number_of_factors):
    factors_heading.append('F' + str(i+1))

variables = standardized_inputs.columns.values
loadings = pd.DataFrame(fa.loadings_, index=variables, columns=factors_heading)

loadings

Explained Variance

In [ ]:
indx = ['SS Loadings', 'Proportion Variance', 'Cumulative Variance']
explained_var = pd.DataFrame(fa.get_factor_variance(), columns=factors_heading, index=indx)
explained_var

Total 55.3% cumulative Variance explained by the 2 factors

### Factored Data

To calculate the values of a principal component, we can define our own function to calculate a principal component given the loadings and the input variables’ values:

In [ ]:
def calcpc(variables, loadings):
    # find the number of samples in the data set and the number of variables
    numsamples, numvariables = variables.shape
    # make a vector to store the component
    pc = np.zeros(numsamples)
    # calculate the value of the component for each sample
    for i in range(numsamples):
        valuei = 0
        for j in range(numvariables):
            valueij = variables.iloc[i, j]
            loadingj = loadings[j]
            valuei = valuei + (valueij * loadingj)
        pc[i] = valuei
    return pc

In [ ]:
# Factor data of the 3 factors

loadings_df = fa.loadings_.reshape(-1,number_of_factors)

pc1_data = calcpc(pd.DataFrame(standardized_inputs), loadings_df[:,0])
pc1_data = pd.DataFrame(pc1_data, columns=['svc_shopping_cpu_use']).round(3)

pc2_data = calcpc(pd.DataFrame(standardized_inputs), loadings_df[:,1])
pc2_data = pd.DataFrame(pc2_data,  columns=['system_inferance_cpu_use']).round(3)

pc3_data = calcpc(pd.DataFrame(standardized_inputs), loadings_df[:,2])
pc3_data = pd.DataFrame(pc3_data,  columns=['svc_shipping_cpu_use']).round(3)

pc4_data = calcpc(pd.DataFrame(standardized_inputs), loadings_df[:,3])
pc4_data = pd.DataFrame(pc4_data,  columns=['svc_payment_cpu_use']).round(3)

targets = pd.DataFrame(targets.values, columns=['orders_ltcy'])

factor_data = pd.concat([pc1_data, pc2_data, pc3_data, pc4_data, targets.round(3)], axis=1, sort=False)
factor_data.shape


In [ ]:
factor_data.head()

In [ ]:
# Save Data to a file
if save:
    factor_data.to_csv(path_or_buf=file_dir + 'order_flow_factored_data.csv', index=False)